In [ ]:
! pip install pennylane-lightning[gpu]

In [2]:
import pennylane as qml
import pennylane.numpy as np
from pennylane.templates import ApproxTimeEvolution

## H2 ground state energy
Following the instruction in Sugisaki *et al.*(2021), we first calculate the H2 ground state energy and judge whether the algorithm runs correctly or not.

In [158]:
dev_H2 = qml.device("default.qubit", wires=5)

def make_hamiltonian_H2(distance):
    symbols, coordinates = (['H', 'H'], np.array([0., 0., -distance/2, 0., 0., distance/2]))
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, 
                                                mapping='jordan_wigner',
                                                wires=[i + 1 for i in range(4)],
                                                active_electrons=2,
                                                active_orbitals=2)
    return H

def Prep_H2():
    qml.CNOT(wires=[0,3])
    qml.CNOT(wires=[0,4])
    angle = np.arccos(np.sqrt(1/2))
    qml.RY(-angle, wires=[2])
    qml.CNOT(wires=[2,1])
    qml.CNOT(wires=[2,3])
    qml.CNOT(wires=[2,4])

def Prep_inv_H2():
    qml.CNOT(wires=[2,4])
    qml.CNOT(wires=[2,3])
    qml.CNOT(wires=[2,1])
    angle = np.arccos(np.sqrt(1/2))
    qml.RY(angle, wires=[2])
    qml.CNOT(wires=[0,4])
    qml.CNOT(wires=[0,3])

@qml.qnode(dev_H2)
def circuit_H2(distance, epsilon, time=0.5, trotterN=5)
    qml.Hadamard(wires=0)
    Prep_H2()
    H = make_hamiltonian_H2(distance)
    ApproxTimeEvolution(H, time, trotterN)
    Prep_inv_H2()
    qml.PhaseShift(epsilon * time, wires=0)
    qml.Hadamard(wires=0)
    return qml.probs(wires=0)

left = -10
right = 10
limit = 0.01

while left + limit < right:
    c1 = left + (right-left)/3
    c2 = right - (right-left)/3
    if circuit_H2(0.74, c1)[0] > circuit_H2(0.74, c2)[0]:
        right = c2
    else:
        left = c1

print("H2 ground state energy : " + str((left + right) / 2) + " Hartree")

H2 ground state energy : -0.9454605707925443 Hartree


The actual value of H2 ground state energy is -1.174476 hartree, therefore the result of this calculation is close to the actual value.  
In the same way, we can calculate the ground state energy of BeH2.  

In [159]:
dev = qml.device("default.qubit", wires=9)
# 0: ancillary, 1 ~ 8: LUMO+1 α β, LUMO α β, ... , HOMO-1 α β

# BeH2
In previous section, we set the prep circuit with the instruction of the paper, but now in calculating BeH2, there is no great idea to prepare the Prep circuit.  

The Prep circuit is applyed to make the qubits express a wave function of ground state. So we can use an optimized VQE ansatz circuit as a Prep circuit instead of a perfect and hard-coded one.  

This is the purpose of this project.

## Making Hamiltonian

In [160]:
def make_hamiltonian(distance):
    symbols, coordinates = (['Be', 'H', 'H'], np.array([0., 0., 0., 0., 0., -distance, 0., 0., distance]))
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, 
                                                mapping='jordan_wigner',
                                                wires=[i for i in range(8)],
                                                active_electrons=4,
                                                active_orbitals=4)
                                                #basis='6-31g') # <--- this is optional.
    return H

## VQE with a few iteration
We run a few iteration VQE to find an optimized ansatz to use as a wave function of the ground state.

In [161]:
def ansatz(params, wires=8):
  for i in range(wires):
    qml.RX(params[0 + 2*i], wires=i+1)
    qml.RY(params[1 + 2*i], wires=i+1)
  for i in range(int(wires/2)):
    qml.CNOT(wires=[2 * i + 1, 2 * i + 2])
  for i in range(int(wires/2) - 1):
    qml.CNOT(wires=[2 * i + 2, 2 * i + 3])

In [162]:
init_params = np.array([np.random.random()]*16, requires_grad=True)

max_iterations = 20
conv_tol = 1e-06
step_size = 0.5

In [163]:
@qml.qnode(dev)
def cost_fn(params):
    ansatz(params)
    return qml.expval(make_hamiltonian(1.3))

In [164]:
opt = qml.GradientDescentOptimizer(stepsize=step_size)

params = init_params

gd_param_history = [params]
gd_cost_history = []

for n in range(max_iterations):

    # Take step
    params, prev_energy = opt.step_and_cost(cost_fn, params)
    gd_param_history.append(params)
    gd_cost_history.append(prev_energy)

    energy = cost_fn(params)

    # Calculate difference between new and old energies
    conv = np.abs(energy - prev_energy)

    if n % 2 == 0:
        print(
            "Iteration = {:},  Energy = {:.8f} Ha,  Convergence parameter = {"
            ":.8f} Ha".format(n, energy, conv)
        )

    if conv <= conv_tol:
        break

print()
print("Final value of the energy = {:.8f} Ha".format(energy))
print("Number of iterations = ", n)

Iteration = 0,  Energy = -13.26771225 Ha,  Convergence parameter = 0.08800343 Ha
Iteration = 2,  Energy = -13.36611490 Ha,  Convergence parameter = 0.03995411 Ha
Iteration = 4,  Energy = -13.41498152 Ha,  Convergence parameter = 0.02064296 Ha
Iteration = 6,  Energy = -13.44277269 Ha,  Convergence parameter = 0.01217530 Ha
Iteration = 8,  Energy = -13.46046127 Ha,  Convergence parameter = 0.00795065 Ha
Iteration = 10,  Energy = -13.47260147 Ha,  Convergence parameter = 0.00554294 Ha
Iteration = 12,  Energy = -13.48132050 Ha,  Convergence parameter = 0.00401730 Ha
Iteration = 14,  Energy = -13.48775076 Ha,  Convergence parameter = 0.00297856 Ha
Iteration = 16,  Energy = -13.49256896 Ha,  Convergence parameter = 0.00223918 Ha
Iteration = 18,  Energy = -13.49621640 Ha,  Convergence parameter = 0.00169891 Ha

Final value of the energy = -13.49770029 Ha
Number of iterations =  19


## Prep Circuit made of Ansatz of VQE
Replaced RX, RY gate on ansatz with Controlled-RX, RY gate, respectively.

In [165]:
def Prep(wires=8):
  for i in range(wires):
    qml.CRX(params[0 + 2*i], wires=[0, i+1])
    qml.CRY(params[1 + 2*i], wires=[0, i+1])
  for i in range(int(wires/2)):
    qml.CNOT(wires=[2 * i + 1, 2 * i + 2])
  for i in range(int(wires/2) - 1):
    qml.CNOT(wires=[2 * i + 2, 2 * i + 3])

def Prep_inv(wires=8):
  for i in reversed(range(int(wires/2) - 1)):
    qml.CNOT(wires=[2 * i + 2, 2 * i + 3])
  for i in reversed(range(int(wires/2))):
    qml.CNOT(wires=[2 * i + 1, 2 * i + 2])
  for i in reversed(range(wires)):
    qml.CRY(-params[1 + 2*i], wires=[0, i+1])
    qml.CRX(-params[0 + 2*i], wires=[0, i+1])


## Apply circuit and optimization

Apply the BPDE circuit in Fig.1 of Sugisaki *et al.*(2021)   
If the epsilon approaches to the exact Energy, the probability of observing |0> on the ancillary qubit (wires=[0]) will become closer to 1.  
Using this "function", we optimize the ground state energy with ternary search algorithm.

In [166]:
@qml.qnode(dev)
def circuit(distance, epsilon, time=0.5, trotterN=5):
    qml.Hadamard(wires=0)
    Prep()
    H = make_hamiltonian(distance)
    ApproxTimeEvolution(H, time, trotterN)
    Prep_inv()
    qml.PhaseShift(epsilon * time, wires=0)
    qml.Hadamard(wires=0)
    return qml.probs(wires=0)

In [167]:
left = -20
right = -10
limit = 0.01

while left + limit < right:
    c1 = left + (right-left)/3
    c2 = right - (right-left)/3
    if circuit(1.3, c1)[0] > circuit(1.3, c2)[0]:
        right = c2
    else:
        left = c1

print("BeH2 ground state energy : " + str((left + right) / 2) + " Hartree")

BeH2 ground state energy : -16.51539291717739 Hartree


### Still have a problem
Theoretically, the probability is independent of the evolution time of Hamiltonian.  
In reality, however, if we change the evolution time, the probability will be change a little as shown below.  
This shows that there is a possibility that the wave function we prepared using an ansatz of VQE is not perfect or that the Hamiltonian is not detailed enough to express BeH2.

In [168]:
print(circuit(1.3, -16.52, time=0.5))
print(circuit(1.3, -16.52, time=2.0))

[0.98242042 0.01757958]
[0.81261332 0.18738668]
